In [6]:
import yaml
import sqlparse
import os
import re

import pandas as pd
os.chdir('Z:\OMOP\omop_etl')
print(os.getcwd())

from omop_etl.datastore import DataStore
from omop_etl.load import Loader
from omop_etl.utils import search
from omop_etl.bo import bo_query, format_stage_query

omop = DataStore('config.yml')

Z:\OMOP\omop_etl


## Configuration files

In [2]:
from omop_etl.config import ProjectConfig, ETLConfig

In [3]:
config = ProjectConfig('config.yml')

In [4]:
# config._config.get('load')

In [5]:
config = ProjectConfig('config.yml')

In [6]:
with config.engine.connect() as con:
    print(con)

In [3]:
etl_config = ETLConfig()

## BO Queries

In [8]:
omop = DataStore('config.yml')

def format_stage_query(dp_name):

    with open('omop_etl/etl_config.yml') as f:
        yml = yaml.safe_load(f) 
    
    aliases= yml['aliases'][dp_name]
    bo_queries = omop.get_bo_query('omop')

    start_date = omop.config_param['date_range']['start_date']
    end_date = omop.config_param['date_range']['end_date']

    sql_query = format_bo_sql(bo_queries[dp_name], dp_name, schema='stage', aliases=aliases)
    patient_id = "select PATIENT_KEY from DWS_OMOP.cohort.PersonList"
    sql_query = sql_query.replace("12345678", patient_id)
    sql_query = sql_query.replace("01/01/1900 00:0:0", start_date)
    sql_query = sql_query.replace("12/31/1900 00:0:0", end_date)

    sql_query = sqlparse.format(sql_query, reindent_aligned=True, indent_with=1)

    return f"EXECUTE ('USE [DWS_PROD];\n {sql_query}')" 

### BO cohort queries

In [5]:
from omop_etl.bo import bo_query
with omop.engine.connect() as con:
    bo_queries = bo_query('cohort_COVID_Broad', con)

len(bo_queries.keys())
# [q for q in bo_queries.keys() if bo_queries[q] == '']

4

In [6]:
bo_queries.keys()

dict_keys(['CPT_CodingDetail', 'ICD_CodingDetail', 'Labs_UFHealth', 'Meds_by_name'])

In [10]:
# bo_queries[]

In [7]:
table = 'CPT_CodingDetail'
# print(format_stage_query(table))
bo_queries[table]

"SELECT  PX_ALL_PATIENTS.PATNT_KEY,  PX_ALL_PATIENTS.PATNT_ID,  PX_ALL_PATIENT_IDENTITIES.IDENT_ID_INT,  PX_JAX_MRN.IDENT_ID_INT FROM  ALL_PATIENTS PX_ALL_PATIENTS RIGHT OUTER JOIN ALL_PATIENT_SNAPSHOTS PX_ALL_PATIENT_SNAPSHOTS ON (PX_ALL_PATIENT_SNAPSHOTS.PATNT_KEY=PX_ALL_PATIENTS.PATNT_KEY)  LEFT OUTER JOIN ALL_PATIENT_IDENTITIES PX_ALL_PATIENT_IDENTITIES ON (PX_ALL_PATIENT_SNAPSHOTS.PATNT_KEY=PX_ALL_PATIENT_IDENTITIES.PATNT_KEY and PX_ALL_PATIENT_IDENTITIES.LOOKUP_IND = 'Y' and PX_ALL_PATIENT_IDENTITIES.IDENT_ID_TYPE = 101)  LEFT OUTER JOIN ALL_PATIENT_IDENTITIES PX_JAX_MRN ON (PX_ALL_PATIENT_SNAPSHOTS.PATNT_KEY=PX_JAX_MRN.PATNT_KEY and PX_JAX_MRN.LOOKUP_IND = 'Y' and PX_JAX_MRN.IDENT_ID_TYPE = 110)  RIGHT OUTER JOIN PROCEDURE_EVENT_DTL ON (PROCEDURE_EVENT_DTL.PATNT_SNAPSHT_KEY=PX_ALL_PATIENT_SNAPSHOTS.PATNT_SNAPSHT_KEY)  LEFT OUTER JOIN ALL_CPT_PROCEDURE_CODES ON (PROCEDURE_EVENT_DTL.CPT_CD_KEY=ALL_CPT_PROCEDURE_CODES.CPT_CD_KEY)  WHERE  (  PROCEDURE_EVENT_DTL.START_DATE BETWEEN '0

In [7]:
with omop.engine.connect() as con:
    bo_queries = bo_query('omop', con)
# print(bo_queries.keys())

# SAVE QUERIES TO FILE
for t in bo_queries.keys():
    try:
        print(t)
        f = open(f'./output/sqlstring_{t.lower()}.sql', 'w')
        f.write(format_stage_query(t))
        format_stage_query(t)
        f.close()
    except ValueError as e:
        print(t)
        raise e

# aliases= yml[table]
# format_bo_sql(bo_queries[table], table, schema='stage')
# omop.execute(format_stage_query('MEASUREMENT_HeartRate'))

# aliases = {}
# for t in bo_queries.keys():
#     with omop.engine.connect() as con:
#         cols = pd.read_sql(f"select top 0 * from stage.{t}", con)
#         aliases[t] = [x.lower() for x in cols.columns]

# with open('col_aliases.yml', 'w') as f:
#     yaml.dump(aliases, f)
#     f.close()

CONDITION


NameError: name 'format_stage_query' is not defined

In [12]:
bo_queries = omop.get_bo_query('omop')
sqlstring = bo_queries['MEASUREMENT_BP_CVP']#'MEASUREMENT_BP_MAP_Cuff'] or 'MEASUREMENT_Height' MEASUREMENT_Weight
parsed = sqlparse.parse(sqlstring)[0]
idx = [parsed.token_index(t) for t in parsed if t.is_keyword and t.value == 'FROM'][0]
columns = parsed.token_prev(idx)[1]

# Extract columns from SELECT clause. If duplicated columns, use alias, 
# else append abbreviated source table name.
colnames = [i.value.split('.')[-1] for i in columns]
dup_cols = set([x for x in colnames if colnames.count(x) > 1])
new_colnames = []

counter = 0
for item in columns:
    if isinstance(item, sqlparse.sql.Identifier):
        colname = item.value.split('.')[-1]
        tabname = item.value.split('.')[-2]
        shrt_tabname = '_'.join([word[:3] for word in tabname.split('_')])
        # print('Identifier', item.value)
    
    elif isinstance(item, sqlparse.sql.Function):
        counter += 1
        fn_name = f'FN_{counter}'
        for x in item:
            if (isinstance(x, sqlparse.sql.Identifier) and x.value == 'cast'):
                print(item.value)
                continue
            else: 
                pass
            if isinstance(x, sqlparse.sql.Parenthesis): 
                for y in x:
                    if isinstance(y, sqlparse.sql.Identifier):
                        print(y.value)
                        print([i.strip() for i in y.value.split(' as ')])
                        # col, dtype = [i.strip() for i in y.value.split('as')]
                        # print(f'try_convert({dtype}, {col})')
    
    # else: 
    #     print('Something else', item.__repr__)

cast(Flowsheet_BP_CVP.MEASR_VALUE as numeric(10,2))
Flowsheet_BP_CVP.MEASR_VALUE as numeric(10,2)
['Flowsheet_BP_CVP.MEASR_VALUE', 'numeric(10,2)']


In [15]:
def flatten(alist):
    flat = []
    while alist: 
        e = alist.pop()
        if type(e) == list: 
            alist.extend(e) 
        else:
            flat.append(e) 
     
    return flat

# def replace_cast_with_try_convert(parsed):    
items = []
for token in parsed.tokens:
    if isinstance(token, sqlparse.sql.IdentifierList) or isinstance(token, sqlparse.sql.Where):
        items = [item for item in token if search('cast', item.value)]
        # print(items)
        for item in items:
            if isinstance(item, sqlparse.sql.Function):
                
                col, dtype = flatten([[i.value.split(' as ') for i in p if isinstance(i, sqlparse.sql.Identifier)] 
                                       for p in item if isinstance(p, sqlparse.sql.Parenthesis)])
                item.value = f'try_convert({dtype},{col})'
                
            else:
                fun_list = flatten([[a for a in i if isinstance(a, sqlparse.sql.Function)] 
                                     for i in item if search('cast', i.value)])
                
                # print(fun_list)
                if not fun_list:
                    fun_list = [i for i in item if (search('cast', i.value) & isinstance(i, sqlparse.sql.Function))]
                
                if not fun_list:
                    fun_list = flatten(flatten([[[a for a in b if isinstance(a, sqlparse.sql.Function)] 
                                                for b in i if isinstance(b, sqlparse.sql.Operation)] for i in item if search('cast', i.value)]))

                # print(fun_list)
                for fun in fun_list:
                    col, dtype = flatten([[i.value.split(' as ') for i in p if isinstance(i, sqlparse.sql.Identifier)] 
                                           for p in fun if isinstance(p, sqlparse.sql.Parenthesis)][0])
                    # print(fun)
                    item.value = item.value.replace(fun.value, f'try_convert({dtype},{col})')
                    # print(item.value)
        
        token.value = ''.join(item.value for item in token)

        print(token.value)

dbo.ALL_PATIENTS.PATNT_KEY,  PATNT_ENCNTR_KEY_XREF1.PATNT_ENCNTR_KEY,  Flowsheet_HTandWT_Height.MEASR_TAKN_DATE,  Flowsheet_HTandWT_Height.MEASR_TAKN_DT,  ( try_convert(Flowsheet_HTandWT_Height.MEASR_VALUE,real) )*2.54,  ALL_PROVIDER_IDS_ATTEND_SER.PROVIDR_KEY,  ALL_PROVIDERS_VISIT.PROVIDR_KEY
WHERE  (  dbo.ALL_PATIENTS.PATNT_KEY IN ( 12345678 )  AND  Flowsheet_HTandWT_Height.MEASR_TAKN_DATE >= '01/01/1900 00:0:0'  AND  Flowsheet_HTandWT_Height.MEASR_TAKN_DATE <= '12/31/1900 00:0:0'  AND  (   ( try_convert(Flowsheet_HTandWT_Height.MEASR_VALUE,real) )*2.54 Is Not Null    OR   try_convert(Flowsheet_HTandWT_Height.MEASR_VALUE,real) Is Not Null   )  AND  dbo.PATIENT_ENCOUNTER_DTL.CALCULATED_ENCNTR_STAT_DESC = 'Complete'  AND  ( dbo.ALL_PATIENTS.TEST_IND='N' )  )


In [28]:
import numpy as np

bo_queries = omop.get_bo_query('omop')
sqlstring = bo_queries['MEASUREMENT_Height']#'MEASUREMENT_BP_MAP_Cuff'] or 'MEASUREMENT_Height' MEASUREMENT_Weight MEASUREMENT_BP_CVP
parsed = sqlparse.parse(sqlstring)[0]

def get_function(item):
    if isinstance(item, (sqlparse.sql.Parenthesis, sqlparse.sql.Operation)):
        return list(filter(None, [get_function(a) for a in item]))
    elif isinstance(item, sqlparse.sql.Function):
        return item

def flatten(lst):
    for el in lst:
        if isinstance(el, list):  
            # recurse
            yield from flatten(el)
        else:
            # generate
            yield el
 

# fun_list = []
for token in parsed.tokens:
    if isinstance(token, (sqlparse.sql.IdentifierList, sqlparse.sql.Where)):
        items = [item for item in token if search('cast', item.value)]
        # print(items)
        for item in items:
            if isinstance(item, sqlparse.sql.Function):
                fun_list = flatten([get_function(item)])
                # print(fun_list)

            else:
                fun_list = flatten(list(filter(None, [get_function(i) for i in item])))
                # print(fun_list)

            for fun in fun_list:
                col, dtype = flatten([[i.value.split(' as ') for i in p if isinstance(i, sqlparse.sql.Identifier)] 
                                        for p in fun if isinstance(p, sqlparse.sql.Parenthesis)][0])
                # print(fun)
                item.value = item.value.replace(fun.value, f'try_convert({dtype},{col})')
    
        token.value = ''.join(item.value for item in token)

        print(token.value)

dbo.ALL_PATIENTS.PATNT_KEY,  PATNT_ENCNTR_KEY_XREF1.PATNT_ENCNTR_KEY,  Flowsheet_HTandWT_Height.MEASR_TAKN_DATE,  Flowsheet_HTandWT_Height.MEASR_TAKN_DT,  ( try_convert(real,Flowsheet_HTandWT_Height.MEASR_VALUE) )*2.54,  ALL_PROVIDER_IDS_ATTEND_SER.PROVIDR_KEY,  ALL_PROVIDERS_VISIT.PROVIDR_KEY
WHERE  (  dbo.ALL_PATIENTS.PATNT_KEY IN ( 12345678 )  AND  Flowsheet_HTandWT_Height.MEASR_TAKN_DATE >= '01/01/1900 00:0:0'  AND  Flowsheet_HTandWT_Height.MEASR_TAKN_DATE <= '12/31/1900 00:0:0'  AND  (   ( try_convert(real,Flowsheet_HTandWT_Height.MEASR_VALUE) )*2.54 Is Not Null    OR   try_convert(real,Flowsheet_HTandWT_Height.MEASR_VALUE) Is Not Null   )  AND  dbo.PATIENT_ENCOUNTER_DTL.CALCULATED_ENCNTR_STAT_DESC = 'Complete'  AND  ( dbo.ALL_PATIENTS.TEST_IND='N' )  )


In [87]:
from functools import reduce

[[[[<Function 'cast(F...' at 0x2BE606D8EC8>]],
  <Function 'cast(F...' at 0x2BE606D89C8>]]

In [24]:
reduce(list.__add__, (list(i) for i in [[i.value.split(' as ') for i in p if isinstance(i, sqlparse.sql.Identifier)] 
                                        for p in fun if isinstance(p, sqlparse.sql.Parenthesis)][0]))

['Flowsheet_BP_CVP.MEASR_VALUE', 'numeric(10,2)']

In [26]:
fun_list

[<Function 'cast(F...' at 0x2BE60C0E2C8>,
 <Name 'cast' at 0x2BE605A7E28>,
 <Punctuation '(' at 0x2BE605A7E88>,
 <Identifier 'Flowsh...' at 0x2BE60C1C8C8>,
 <Punctuation ')' at 0x2BE605D31C8>]

In [14]:
for token in parsed.tokens:
#     print(token.ttype)
    if isinstance(token, sqlparse.sql.IdentifierList) or isinstance(token, sqlparse.sql.Where):
        print(token.ttype)

None
None


In [2]:
from omop_etl.stage import Stager
stage = Stager('config.yml')
# print(l.update_mappings('person'))
# print(l.update_mappings('visit_occurrence'))

# print(l.preload_all())

# print(l.load_table('person'))
# print(l.load_table('death'))
# print(l.load_table('condition_occurrence'))
# print(l.load_table('procedure_occurrence'))
# print(l.load_table('drug_exposure'))
# print(l.load_table('observation'))
# print(l.load_table('provider'))
# print(l.load_table('care_site'))
# print(l.load_table('location'))

In [5]:
load_conf = stage.store.config_param['load']
# stage.stage_table('measurement','res_fio2')

In [15]:
for t in load_conf.keys():
    if load_conf[t]:
        for part in load_conf[t].keys():
            stage.stage_table(t, part)
    else:
        if t not in ('provider','care_site','location'): 
            stage.stage_table(t)

no parts person
no parts death
no parts condition_occurrence
has part procedure_occurrence icd
has part procedure_occurrence cpt
has part drug_exposure order
has part drug_exposure admin
has part measurement bp
has part measurement heart_rate
has part measurement height
has part measurement lab
has part measurement pain
has part measurement qtcb
has part measurement res_dev
has part measurement res_etco2
has part measurement res_fio2
has part measurement res_gcs
has part measurement res_o2
has part measurement res_peep
has part measurement res_pip
has part measurement res_resp
has part measurement res_spo2
has part measurement res_tidal
has part measurement res_vent
has part measurement rothman
has part measurement sofa
has part measurement temp
has part measurement weight
has part observation icu
has part observation lda
has part observation vent
has part observation payer
has part observation smoking
has part observation zipcode
no parts visit


In [131]:
import itertools
import re 

for token in parsed.tokens:
    if isinstance(token, sqlparse.sql.Where):
        print([item for item in token if search('cast', item.value)])

        # par = list(itertools.chain(*token))
        # for p in par:
        #     print(p.__repr__)
            # if isinstance(i, sqlparse.sql.Parenthesis):
            #     # defs = extract_definitions(i)
            #     for x in i:
            #         if isinstance(x, sqlparse.sql.Parenthesis):
            #             for y in x:
            #                 print(y.__repr__)

[<Parenthesis '( dbo...' at 0x2A1B00F7AC8>]


In [79]:
import sqlparse

def extract_definitions(token_list):
    # assumes that token_list is a parenthesis
    definitions = []
    tmp = []
    par_level = 0
    for token in token_list.flatten():
        if token.is_whitespace:
            continue
        elif token.match(sqlparse.tokens.Punctuation, '('):
            par_level += 1
            continue
        if token.match(sqlparse.tokens.Punctuation, ')'):
            if par_level == 0:
                break
            else:
                par_level += 1
        elif token.match(sqlparse.tokens.Punctuation, ','):
            if tmp:
                definitions.append(tmp)
            tmp = []
        else:
            tmp.append(token)
    if tmp:
        definitions.append(tmp)
    return definitions